%%% Imports

In [1]:

from __future__ import annotations
from time import time
from QCRVPRDataset import QCRVPRDataset
from VPRNetwork import VPRNetwork
from plotting import plot_confusion_matrix, plot_match_images, plot_gps
from constants import brisbane_event_traverses, qcr_traverses, brisbane_event_traverses_aliases
from utils import get_short_traverse_name

import os, sys
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy import signal
from datetime import datetime

import numpy as np

import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.classifier import Rate


In [2]:
# Training settings
from cgi import test


epochs = 50
batch_size = 10

# Network settings
input_size = 34
threshold = 1.0

# Data settings
train_traverse = qcr_traverses[0]
test_traverse = qcr_traverses[3]
train_name = get_short_traverse_name(train_traverse)
test_name = get_short_traverse_name(test_traverse)
num_places = 30
start_time = 0
place_gap = 5 #164/num_places # The streams run for approximately 164 seconds 
samples_per_sec = 1000
place_duration = 2
max_spikes = 2000
incorporate_speed = True


# Sequencer settings
sequence_length = 3

# Plot settings
redo_plot = False
vmin = 0
vmax = 0.05

def transpose( matrix):
    if len(matrix) == 0:
        return []
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]


# Make a folder for the trained network
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Use GPU
print(torch.cuda.is_available())
device = torch.device('cuda')

#---------------- Create the Network -----------------#

# Create the network
net = VPRNetwork(input_size, num_places, threshold=threshold).to(device)


True


In [3]:
# Load the data
training_set = QCRVPRDataset(train_traverse, train=True, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time,samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)
if incorporate_speed:
    testing_set  = QCRVPRDataset(test_traverse, train=False, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size, relative_place_times=training_set.relative_place_times, training_duration=training_set.training_duration) # training_duration=training_set.training_duration
else:
    testing_set  = QCRVPRDataset(test_traverse, train=False, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size, relative_place_times=training_set.relative_place_times)

train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=True)

Loading training event streams ...


  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 164.20s (which is 36141275 events)


/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i


[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145]
The number of training substreams is: 30


...

  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 71.37s (which is 37397552 events)
[ 0.          2.1732159   4.3464318   6.51964771  8.69286361 10.86607951
 13.03929541 15.21251131 17.38572721 19.55894312 21.73215902 23.90537492
 26.07859082 28.25180672 30.42502263 32.59823853 34.77145443 36.94467033
 39.11788623 41.29110213 43.46431804 45.63753394 47.81074984 49.98396574
 52.15718164 54.33039755 56.50361345 58.67682935 60.85004525 63.02326115]
Place duration 2
Speed ratio 0.4346431803636916


/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i


The number of testing substreams is: 30


/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['t'] -= chop_start


In [4]:
# Define an optimiser 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training the network
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

# Create a training assistant object
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)


if redo_plot:
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)

else: 
    for epoch in range(epochs):

        for i, (input, label) in enumerate(train_loader): # training loop
            output = assistant.train(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        for i, (input, label) in enumerate(test_loader): # training loop
            output = assistant.test(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if epoch%20 == 19: # cleanup display
            print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
            stats_str = str(stats).replace("| ", "\n")
            print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')

        if stats.testing.best_accuracy:
            torch.save(net.state_dict(), trained_folder + '/network.pt')
        stats.update()
        stats.save(trained_folder + '/')
        net.grad_flow(trained_folder + '/')

[Epoch  0/50] Train loss =     0.54516                        accuracy = 0.03333 | Test  loss =     0.55948                        accuracy = 0.03333
Accuracy was none
[Epoch  1/50] Train loss =     0.44821 (min =     0.54516)    accuracy = 0.06667 (max = 0.03333) | Test  loss =     0.38524 (min =     0.55948)    accuracy = 0.03333 (max = 0.03333)
Accuracy was none
[Epoch  2/50] Train loss =     0.36391 (min =     0.44821)    accuracy = 0.40000 (max = 0.06667) | Test  loss =     0.40895 (min =     0.38524)    accuracy = 0.06667 (max = 0.03333)
Accuracy was none
[Epoch  3/50] Train loss =     0.29485 (min =     0.36391)    accuracy = 0.80000 (max = 0.40000) | Test  loss =     0.34425 (min =     0.38524)
Accuracy was none

Accuracy was none
[Epoch  4/50] Train loss =     0.24264 (min =     0.29485)    accuracy = 0.93333 (max = 0.80000) | Test  loss =     0.38477 (min =     0.34425)    accuracy = 0.03333 (max = 0.06667)
Accuracy was none
[Epoch  5/50] Train loss =     0.21649 (min =     0

In [5]:
# import the best network during training 
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Get the output for the input to each place
test_loader2  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=False)
rate = []
labels = []
for i, (input, label) in enumerate(test_loader2):
    output = net(input.to(device)) # Get network output
    #guesses = assistant.classifier(output).cpu().data.numpy() # get the predictions 
    rate.extend(Rate.rate(output).cpu().data.numpy()) # Get the firing rates for each place 
    labels.extend(label.cpu().data.numpy()) # Get place labels


# # Get rates in percentages of total
# for i in range(num_places):
#     sum = np.sum(rate[i])
#     if sum != 0:
#         rate[i] = np.divide(rate[i],sum)

# Make confusion matrix annotations
accuracy = 0
matches = []
annotations = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(rate[qryIndex])
    matches.append(max_idx)
    annotations[max_idx][qryIndex] = 'x'
    if max_idx ==qryIndex:
        accuracy += 1


#--------------- Apply a sequencer ------------------#
I = np.identity(sequence_length)
conv = signal.convolve2d(rate, I, mode='same')
print(np.shape(conv))

# Make confusion matrix annotations
accuracy_s = 0
matches_with_seq = []
annotations_s = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(conv[qryIndex])
    matches_with_seq.append(max_idx)
    annotations_s[max_idx][qryIndex] = 'x'
    if max_idx == qryIndex:
        accuracy_s += 1



(30, 30)


In [6]:

#--------------- Save Results ------------------#

# Make new folder for results
from tracemalloc import start
from plotting import plot_gps


time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_path = "./../results/" + time_stamp
os.mkdir(results_path)


# Save query and match images 
images_path = results_path + "/matched_images"
images_path_seq = results_path + "/matched_images_seq"
os.mkdir(images_path)
os.mkdir(images_path_seq)

print('help')
plot_match_images(images_path + "/", matches, training_set.place_images, testing_set.place_images)
plot_match_images(images_path_seq + "/", matches_with_seq, training_set.place_images, testing_set.place_images)

# Save the confusion matrices
confusion_path = results_path + "/confusion_matrices"
os.mkdir(confusion_path)
rate = transpose(rate)
conv = transpose(conv)
output_path = confusion_path + "/confusion_matrix" 
output_path_s = confusion_path + "/confusion_matrix_seq" 
plot_confusion_matrix(rate, labels, annotations, output_path, vmin, vmax)
plot_confusion_matrix(conv, labels, annotations_s, output_path_s, vmin, vmax)

# Save GPS map'
# gps_path = results_path + "/gps_locations"
# plot_gps(gps_path, training_set.place_locations, testing_set.place_locations)

# Save test settings and accuracy
accuracy = accuracy/num_places
accuracy_s = accuracy_s/num_places


help


<Figure size 1296x288 with 0 Axes>

<Figure size 1296x288 with 0 Axes>

In [7]:

log_path = results_path + "/log.txt"
log_string  = """
---- DATA SETTINGS ---- 
Training datasets = {}
Testing datasets = {} 
# Places = {}
Start time = {} [s]
Place gap = {} [s]
Place duration = {} [s]
Samples per second = {} 
Max spikes per sample = {}
---- NETWORK SETTINGS ---- 
Input size = {}x{} 
Threshold = {}
---- TRAINING SETTINGS ---- 
Epochs = {}
Batch size = {}
---- SEQUENCER SETTINGS ----
Sequence lengtth = {} 
---- RESULTS ----
Accuracy = {}
Accuracy (sequencer) = {}
""".format(train_name,test_name,num_places,start_time,place_gap,place_duration,
                samples_per_sec,max_spikes,input_size,input_size,threshold,epochs,
                batch_size,sequence_length,accuracy,accuracy_s)
f = open(log_path,'w')
f.write(log_string)
f.close()

print("The accuracy of the network is: " + str(accuracy))
print("The accuracy with a sequencer is: " + str(accuracy_s))

The accuracy of the network is: 0.06666666666666667
The accuracy with a sequencer is: 0.1
